<a href="https://colab.research.google.com/github/Axel02leon/Intro-to-Machine-Learning-/blob/main/HW7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem 1A

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, confusion_matrix
import time


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32,padding =4),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [ ]:
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transforms.ToTensor(), download=True)

100%|██████████| 170M/170M [00:18<00:00, 9.00MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
batch_size = 64
train_Loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_Loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Definding the CNN Model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)


    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [ ]:
# Training Function
def train_model(model, criterion, optimizer, train_loader, val_loader, n_epochs = 200, patience=300):
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(1, n_epochs + 1):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Calculate average training loss
        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation step
        val_loss, val_accuracy = evaluate_model(model, val_loader, return_loss=True)

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping triggered at epoch {epoch}")
                break

        val_losses.append(val_loss)

        print(f"Epoch {epoch}/{n_epochs} - Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

    return train_losses, val_losses


In [ ]:
# Evaluation Function
def evaluate_model(model, loader, return_loss=False):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    criterion = nn.CrossEntropyLoss()
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            if return_loss:
                total_loss += criterion(outputs, targets).item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == targets).sum().item()
            total += targets.size(0)
            true_labels.extend(targets.cpu().numpy())
            predicted_labels.extend(predicted.cpu().numpy())

    accuracy = 100 * correct / total
    if return_loss:
        average_loss = total_loss / len(loader)
        return average_loss, accuracy
    else:
        return accuracy


In [ ]:
# Instantiate Model, Loss Function, and Optimizer
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the Model
n_epochs = 200
train_losses, val_losses = train_model(model, criterion, optimizer, train_Loader, test_Loader, n_epochs)

# Test the Model
test_accuracy = evaluate_model(model, test_Loader)
print(f"Test Accuracy: {test_accuracy:.2f}%")

Epoch 1/200 - Train Loss: 1.5454, Val Loss: 1.2110, Val Acc: 56.16%
Epoch 2/200 - Train Loss: 1.1847, Val Loss: 1.0677, Val Acc: 62.18%
Epoch 3/200 - Train Loss: 1.0400, Val Loss: 0.9934, Val Acc: 65.15%
Epoch 4/200 - Train Loss: 0.9369, Val Loss: 0.8952, Val Acc: 68.78%
Epoch 5/200 - Train Loss: 0.8581, Val Loss: 0.8776, Val Acc: 69.61%
Epoch 6/200 - Train Loss: 0.7935, Val Loss: 0.8439, Val Acc: 70.52%
Epoch 7/200 - Train Loss: 0.7292, Val Loss: 0.8127, Val Acc: 71.85%
Epoch 8/200 - Train Loss: 0.6780, Val Loss: 0.8348, Val Acc: 71.58%
Epoch 9/200 - Train Loss: 0.6338, Val Loss: 0.8245, Val Acc: 72.06%
Epoch 10/200 - Train Loss: 0.5827, Val Loss: 0.8102, Val Acc: 73.11%
Epoch 11/200 - Train Loss: 0.5429, Val Loss: 0.8409, Val Acc: 72.63%
Epoch 12/200 - Train Loss: 0.4996, Val Loss: 0.8594, Val Acc: 73.11%
Epoch 13/200 - Train Loss: 0.4637, Val Loss: 0.8430, Val Acc: 73.52%
Epoch 14/200 - Train Loss: 0.4306, Val Loss: 0.8824, Val Acc: 73.23%
Epoch 15/200 - Train Loss: 0.4013, Val Loss

Problem B1
# New Section

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, confusion_matrix
import time


In [ ]:
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Data Preprocessing
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
# Load CIFAR-10 Dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

# Create DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


100%|██████████| 170M/170M [00:13<00:00, 13.1MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
for inputs, targets in train_loader:
    print(f"Sample Input Shape: {inputs.shape}")  # Expected: [batch_size, 3, 32, 32]
    print(f"Sample Target Shape: {targets.shape}")  # Expected: [batch_size]
    break

Sample Input Shape: torch.Size([64, 3, 32, 32])
Sample Target Shape: torch.Size([64])


In [ ]:
# Extended CNN Model
class CNNEXTEN(nn.Module):
    def __init__(self):
        super(CNNEXTEN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [ ]:
# Training Function
def train_model(model, criterion, optimizer, train_loader, val_loader, n_epochs=200):
    train_losses = []
    val_losses = []

    for epoch in range(1, n_epochs + 1):
        model.train()
        running_loss = 0.0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Calculate average training loss
        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation step
        val_loss, val_accuracy, _, _ = evaluate_model(model, val_loader)  # Unpack only needed values
        val_losses.append(val_loss)

        # Print training and validation results every epoch
        print(f"Epoch {epoch}/{n_epochs} - Train Loss: {avg_train_loss:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

    return train_losses, val_losses


In [ ]:
# Evaluation Function with F1-Score, Confusion Matrix, and Accuracy
def evaluate_model(model, loader):
    model.eval()
    all_targets = []
    all_predictions = []
    correct = 0
    total = 0
    criterion = nn.CrossEntropyLoss()
    total_loss = 0.0

    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()

            _, predictions = torch.max(outputs, 1)
            correct += (predictions == targets).sum().item()
            total += targets.size(0)

            all_targets.extend(targets.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())

    # Calculate metrics
    accuracy = correct / total * 100
    f1 = f1_score(all_targets, all_predictions, average='weighted')
    cm = confusion_matrix(all_targets, all_predictions)
    avg_loss = total_loss / len(loader)

    # Print metrics
    print(f"Evaluation Results:")
    print(f"Average Loss: {avg_loss:.4f}")
    print(f"Accuracy: {accuracy:.2f}%")
    print(f"F1 Score: {f1:.4f}")
    print(f"Confusion Matrix:\n{cm}")

    return avg_loss, accuracy, f1, cm



In [ ]:
# Train and Test Extended CNN
extended_model = CNNEXTEN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(extended_model.parameters(), lr=0.001)

print("\nTraining Extended CNN...")
train_losses, val_losses = train_model(extended_model, criterion, optimizer, train_loader, test_loader, n_epochs=200)

extended_test_accuracy = evaluate_model(extended_model, test_loader)
print(f"Extended CNN Test Accuracy: {extended_test_accuracy:.2f}%")
print("\nEvaluating Extended CNN...")
_, test_accuracy, f1, cm = evaluate_model(extended_model, test_loader)



Training Extended CNN...
Evaluation Results:
Average Loss: 1.3022
Accuracy: 52.82%
F1 Score: 0.5172
Confusion Matrix:
[[696  52  57  11  15  21  16   8  73  51]
 [ 31 713   6   8   7  14  14   6  28 173]
 [118  15 267  49 204 147 123  29  21  27]
 [ 30  26  81 206  85 348 125  41  14  44]
 [ 53  16  89  39 434  83 168  90  11  17]
 [ 13   8  82  89  74 606  47  47  10  24]
 [ 12  10  31  52  94  38 710  15  10  28]
 [ 29  14  36  31  89 178  27 520   7  69]
 [277  90  21  12   4  17  18   4 502  55]
 [ 37 198  12  10   7  26  23  14  45 628]]
Epoch 1/200 - Train Loss: 1.6091, Val Loss: 1.3022, Val Accuracy: 52.82%
Evaluation Results:
Average Loss: 1.0624
Accuracy: 61.61%
F1 Score: 0.6148
Confusion Matrix:
[[757  34  54  11  18   6   6  21  47  46]
 [ 31 779   2   9   4   4  11  12  10 138]
 [101  11 406  77 166 111  69  37   9  13]
 [ 34  12  65 405  91 241  79  42  14  17]
 [ 41   6  87  75 576  49  71  88   5   2]
 [ 15   7  63 170  60 569  23  73   7  13]
 [ 12  10  43 112  99  17 

TypeError: unsupported format string passed to tuple.__format__